### Sprint 4 – Notebook 04: Análisis de Slices Problemáticos

Este notebook analiza el rendimiento del modelo en diferentes **subpoblaciones (“slices”)**.

Incluye:

- Carga de resultados generados por scripts del Sprint 4  
  - `slices_problematicos.csv`
  - `slices_problematicos_ic.csv`
- Cálculo y ranking de slices más riesgosos
- Visualizaciones comparativas
- Comentarios técnicos para el informe Sprint 4

Rutas ajustadas para ejecución desde:
`notebooks/Sprint4/`


In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set(style="whitegrid")


In [ ]:
SLICE_PATH = Path("../../models/sprint4/slices_problematicos.csv")
SLICE_IC_PATH = Path("../../models/sprint4/slices_problematicos_ic.csv")

SLICE_PATH, SLICE_IC_PATH


In [ ]:
if not SLICE_PATH.exists():
    raise FileNotFoundError(f"No se encontró {SLICE_PATH}. Ejecuta 02_SlicesProblematicos.py")

df_slices = pd.read_csv(SLICE_PATH)

print("Datos cargados:")
df_slices.head()


In [ ]:
if SLICE_IC_PATH.exists():
    df_ic = pd.read_csv(SLICE_IC_PATH)
    print("IC encontrados:")
    display(df_ic.head())
else:
    df_ic = None
    print("⚠ No hay archivo slices_problematicos_ic.csv")


In [ ]:
df_sorted = df_slices.sort_values(by="f1_score", ascending=True)
df_sorted.head(10)


In [ ]:
top_n = 10
df_plot = df_sorted.head(top_n)

plt.figure(figsize=(10, 6))
sns.barplot(
    data=df_plot,
    x="f1_score",
    y="slice_name",
    palette="Reds_r"
)

plt.title(f"Top {top_n} slices más problemáticos (F1 bajo)")
plt.xlabel("F1-score")
plt.ylabel("Slice")
plt.tight_layout()
plt.show()


In [ ]:
if "precision" in df_slices.columns and "recall" in df_slices.columns:
    plt.figure(figsize=(12,6))
    sns.scatterplot(
        data=df_slices,
        x="precision",
        y="recall",
        size="support",
        hue="f1_score",
        palette="viridis",
        sizes=(20,300)
    )

    plt.title("Precisión vs Recall por Slice (tamaño ∝ soporte)")
    plt.xlabel("Precisión")
    plt.ylabel("Recall")
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No existen columnas 'precision' y 'recall' en slices.")


In [ ]:
if df_ic is not None:
    df_merge = df_slices.merge(df_ic, on="slice_name", how="left")
    df_merge.head()
else:
    df_merge = df_slices.copy()


In [ ]:
if df_ic is not None:
    plt.figure(figsize=(10, 6))
    sns.pointplot(
        data=df_merge.head(top_n),
        x="f1_score",
        y="slice_name",
        join=False,
        capsize=0.4,
        errwidth=1,
        color="blue"
    )
    plt.title("F1-score por Slice con Intervalos de Confianza")
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No se pueden graficar intervalos de confianza")


### Diagnóstico técnico de slices problemáticos

Los slices con bajo desempeño en F1 suelen deberse a:

1. **Pocos ejemplos (support bajo)** → el modelo no generaliza bien.  
2. **Alta variabilidad** → datos inconsistentes o ruidosos.  
3. **Distribución distinta al dataset global** → covariate shift.  
4. **Atributos específicos del slice** que afectan la separabilidad:
   - Obras pequeñas o muy grandes  
   - Empresas con poco historial  
   - Funcionarios con perfil atípico  

Se deben revisar:

- Features del slice  
- Ejemplos de entrenamiento  
- Ejemplos del conjunto de validación  
- Diferencias con la distribución global  


In [ ]:
recs = []

for _, row in df_sorted.head(5).iterrows():
    recs.append({
        "slice": row["slice_name"],
        "support": row.get("support", "NA"),
        "f1_score": row["f1_score"],
        "recomendacion": f"Revisar casos del slice '{row['slice_name']}'. \
Puede requerir oversampling, ingeniería de atributos o entrenamiento específico."
    })

pd.DataFrame(recs)


### Resumen del análisis de slices

- Se identificaron las subpoblaciones con menor desempeño del modelo.
- Los slices ordenados por *F1-score* permiten priorizar intervenciones.
- Si hay intervalos de confianza, ayudan a evaluar estabilidad del modelo.
- Los casos problemáticos pueden guiar:
  - Oversampling
  - Re-entrenamiento focalizado
  - Mayor ingeniería de atributos
  - Revisión de outliers

Este análisis alimenta directamente el informe Sprint 4 y la toma de decisiones del área técnica.
